In [1]:
!pip install -U \
torch \
transformers \
accelerate \
bitsandbytes \
sentence-transformers \
langchain \
langchain-community \
langchain-text-splitters \
chromadb \
pymupdf \
tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.5 MB/s eta

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import chromadb
import sentence_transformers

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch: 2.9.1+cu128
CUDA available: True


In [5]:
from huggingface_hub import login
login()


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [9]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdf_path = "/content/merck.pdf"

loader = PyMuPDFLoader(pdf_path)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=512,
    chunk_overlap=50
)

chunks = splitter.split_documents(docs)
print("Total chunks:", len(chunks))


Total chunks: 8308


In [10]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
import os

embedding_model = SentenceTransformerEmbeddings(
    model_name="BAAI/bge-large-en-v1.5"
)

persist_dir = "merck_db"
os.makedirs(persist_dir, exist_ok=True)

vectorstore = Chroma.from_documents(
    chunks,
    embedding_model,
    persist_directory=persist_dir
)


/tmp/ipython-input-2167806645.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [11]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "lambda_mult": 0.25}
)


In [52]:
def build_prompt(context, question):
    return f"""[INST]
You are a medical assistant.
Answer strictly using the provided context.
If the answer is not present in the context, say "I don't know."

Context:
{context}

Question:
{question}
[/INST]
"""


In [64]:
SIMILARITY_THRESHOLD = 0.4   # tune between 0.35–0.45 if needed
MAX_NEW_TOKENS = 256
def generate_rag_response(question):
    # 1️ Retrieve documents WITH similarity scores
    results = vectorstore.similarity_search_with_score(
        question,
        k=3
    )

    # 2️ Confidence / hallucination guardrail
    if len(results) == 0 or results[0][1] < SIMILARITY_THRESHOLD:
        return "I don't know."

    # 3️ Build context
    docs = [doc for doc, score in results]
    context = "\n\n".join(d.page_content for d in docs)

    # 4️ Build prompt
    prompt = build_prompt(context, question)

    # 5️ Tokenize
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # 6️ Generate answer (GPU)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            temperature=0.1,
            do_sample=False
        )

    # 7️ REMOVE PROMPT TOKENS (fixes big output issue)
    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
    answer = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return answer.strip()


In [65]:
query = "What is the protocol for managing sepsis in a critical care unit?"
print(generate_rag_response(query))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The protocol for managing sepsis in a critical care unit includes the following steps:

1. Prompt empiric antibiotic therapy: After taking specimens for Gram stain and culture, administer antibiotics immediately based on the suspected source, clinical setting, and previous culture results.
2. Fluid resuscitation: Administer 0.9% saline until Central Venous Pressure (CVP) reaches 8 mm Hg (10 cm H2O) or Pulmonary Artery Occusion Pressure (PAOP) reaches 12 to 15 mm Hg. Oliguria with hypotension is not a contraindication to vigorous fluid resuscitation.
3. Vasopressor support: If a patient remains hypotensive after CVP or PAOP has been raised to target levels, administer dopamine to increase mean arterial pressure to at least 60 mm Hg. If dopamine dose exceeds 20 μg/kg/min, add another vasopressor, typically norepinephrine.
4. Oxygen therapy: Provide oxygen through a mask or nasal prongs, and consider tr


In [66]:
query = "What is your college?"
print(generate_rag_response
(query))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The context does not provide information about the colleges attended by the individuals listed. Therefore, I don't know.
